# Visual DAA Bounding Box Tuning
## Setup

In [1]:
import ipywidgets as widgets
from ipywidgets import interact
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2
from mpl_toolkits.axes_grid1 import ImageGrid
import json

#### Data and Image Import

In [2]:
df = pd.read_csv('../datasets/state_data.csv')
sh, sw, _ = cv2.imread('../datasets/imgs/0.jpg').shape
data_ims = np.zeros((50, sh, sw, 3), dtype=int)
# load hfov and vfov
with open('metadata.json','r') as datafile:
    metadata = json.load(datafile)
hfov = metadata["hfov"]
vfov = metadata["vfov"]
for i in range(10):
    data_ims[i, :, :, :] = np.array(cv2.cvtColor(cv2.imread(
        '../datasets/imgs/' + str(i) + '.jpg'), cv2.COLOR_BGRA2BGR)[:, :, ::-1])

#### Utility Functions

In [3]:
def get_bb_size(e0, n0, u0, e1, n1, u1, aw0=0, daw=1):
    x = n1 - n0
    y = -(e1 - e0) # right-handed coordinates
    z = u1 - u0
    print(f"{x}, {y}, {z}")
    
    # Get height and width of bounding box
    r = np.sqrt(x ** 2 + y ** 2 + z ** 2)
    w = daw * (1 / r) + aw0
    h = (3 / 8) * w
    print(f"{w}, {h}")
    return h, w

In [4]:
def show_img_w_bb(im, xp, yp, w, h):
    f, ax = plt.subplots()
    f.set_figwidth(14)
    f.set_figheight(14)

    ax.imshow(im)

    rect = patches.Rectangle((xp - (w / 2), yp - (h / 2)), w, h, linewidth=1, edgecolor='r', facecolor='none')
    ax.add_patch(rect)

## View and Tune Bounding Boxes
aw0 and daw tune the size of the bounding box

In [6]:
def data_interact_bb(i, aw0, daw, offset):
    xp, yp = df['intr_x'][i], df['intr_y'][i]+offset
    h, w = get_bb_size(df['e0'][i], df['n0'][i], df['u0'][i], df['e1'][i], df['n1'][i], df['u1'][i], aw0=aw0, daw=daw)
    show_img_w_bb(data_ims[i, :, :, :], xp, yp, w, h)

interact(data_interact_bb,
         aw0=widgets.FloatSlider(min=-10, max=10, step=1, value=0),
         daw=widgets.FloatSlider(min=13000, max=18000, step=200, value=18000),
         offset=widgets.FloatSlider(min=-100, max=100, step=1, value=-36),
         i=widgets.IntText(value=0))

interactive(children=(IntText(value=0, description='i'), FloatSlider(value=0.0, description='aw0', max=10.0, m…

<function __main__.data_interact_bb(i, aw0, daw, offset)>

#### Graveyard

In [81]:
    
'''
def get_bounding_box2(e0, n0, u0, h0, e1, n1, u1, p0, tilt=0, hfov=80, vfov=50.25, offset=0, aw0=0, daw=1):

    # Make ownship be the origin
    x = n1 - n0
    y = -(e1 - e0) # right-handed coordinates
    z = u1 - u0

    # Rotate x and y according to ownship heading
    xrot = x * cosd(h0) - y * sind(h0)
    yrot = -(x * sind(h0) + y * cosd(h0))

    # Account for offset
    z = z + offset

    # Rotate z according to tilt angle
    xcam = xrot * cosd(tilt) - z * sind(tilt)
    ycam = yrot
    zcam = xrot * sind(tilt) + z * cosd(tilt)

    # https://www.youtube.com/watch?v=LhQ85bPCAJ8
    xp = ycam / (xcam * tand(hfov / 2))
    yp = zcam / (xcam * tand(vfov / 2))

    # Get xp and yp between 0 and 1
    xp = (xp + 1) / 2
    yp = (yp + 1) / 2

    # Map to pixel location
    xp = xp * sw
    yp = (1 - yp) * sh
    
    print(f"{x}, {y}, {z}")

    # Get height and width of bounding box
    r = np.sqrt(x ** 2 + y ** 2 + z ** 2)
    w = daw * (1 / r) + aw0
    h = (3 / 8) * w
    print(f"{w}, {h}")

    # print(r)
    #print(f"{xp}, {yp}, {w}, {h}")
    return xp, yp, w, h'''

'\ndef get_bounding_box2(e0, n0, u0, h0, e1, n1, u1, p0, tilt=0, hfov=80, vfov=50.25, offset=0, aw0=0, daw=1):\n\n    # Make ownship be the origin\n    x = n1 - n0\n    y = -(e1 - e0) # right-handed coordinates\n    z = u1 - u0\n\n    # Rotate x and y according to ownship heading\n    xrot = x * cosd(h0) - y * sind(h0)\n    yrot = -(x * sind(h0) + y * cosd(h0))\n\n    # Account for offset\n    z = z + offset\n\n    # Rotate z according to tilt angle\n    xcam = xrot * cosd(tilt) - z * sind(tilt)\n    ycam = yrot\n    zcam = xrot * sind(tilt) + z * cosd(tilt)\n\n    # https://www.youtube.com/watch?v=LhQ85bPCAJ8\n    xp = ycam / (xcam * tand(hfov / 2))\n    yp = zcam / (xcam * tand(vfov / 2))\n\n    # Get xp and yp between 0 and 1\n    xp = (xp + 1) / 2\n    yp = (yp + 1) / 2\n\n    # Map to pixel location\n    xp = xp * sw\n    yp = (1 - yp) * sh\n    \n    print(f"{x}, {y}, {z}")\n\n    # Get height and width of bounding box\n    r = np.sqrt(x ** 2 + y ** 2 + z ** 2)\n    w = daw * (1 

In [82]:
def cosd(x):
    return np.cos(x * np.pi / 180)

def sind(x):
    return np.sin(x * np.pi / 180)

def tand(x):
    return np.tan(x * np.pi / 180)